In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from deepface import DeepFace as df
from retinaface import RetinaFace as m
from PIL import Image
import imagehash
import pandas as pd
from pymongo import MongoClient
from deepface.commons import functions
from tqdm import tqdm

In [ ]:

import numpy as np
from datetime import datetime

class VideoCaptureManager:
    def __init__(self, frame_size=(640, 480), codec='XVID'):
        self.frame_size = frame_size
        self.codec = codec
        self.is_capturing = False
        self.capture_duration = 0
        self.frames_buffer = []  # Store frames in a temporary buffer

    def start_capture(self, capture_duration):
        self.capture_duration = capture_duration
        self.is_capturing = True

        # Open the default camera (camera index 0)
        cap = cv2.VideoCapture(0)

        # Add the namedWindow line
        cv2.namedWindow('Video Capture', cv2.WINDOW_NORMAL)

        start_time = cv2.getTickCount()
        while self.is_capturing:
            ret, frame = cap.read()

            if not ret:
                print("Failed to capture video.")
                break

            self.frames_buffer.append(frame)

            # Add the namedWindow line
            cv2.imshow('Video Capture', frame)

            # Bring the window to the front
            cv2.setWindowProperty('Video Capture', cv2.WND_PROP_TOPMOST, 1)

            # Break the loop if 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.stop_capture()

            elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
            if elapsed_time >= self.capture_duration:
                self.stop_capture()

        # Release resources after capturing is done
        cap.release()
        cv2.destroyAllWindows()

    def stop_capture(self):
        self.is_capturing = False

    def read_frames_from_buffer(self):
        return self.frames_buffer

if __name__ == "__main__":
    video_manager = VideoCaptureManager()

    while True:
        user_input = input("Press 's' to start capture, 'q' to quit: ")

        if user_input == 's':
            capture_duration = int(input("Enter capture duration (in seconds): "))
            video_manager.start_capture(capture_duration)
        elif user_input == 'q':
            break
        else:
            print("Invalid input. Press 's' to start capture, 'q' to quit.")

    frames_arr = video_manager.read_frames_from_buffer()

    if frames_arr:
        print(f"Number of frames: {len(frames_arr)}")
    else:
        print("No frames captured.")


In [ ]:
image_directory = "Data\known_faces"

file_list = os.listdir(image_directory)

paths = [os.path.join(image_directory, file) for file in file_list]
print(paths[0])

names = []
for path in paths:

    name=path.split('\\')
    names.append(name[-1])
#print()
print(names[0])

known_faces = []
for i in range(len(names)):
    known_faces.append([paths[i], names[i]])
print()
for i in range(len(known_faces)):
    print(known_faces[i][1])

In [ ]:
frames = []
frames.append(frames_arr[0])
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(1, len(frames_arr)):
    
    
    
    keypoints1, descriptors1 = orb.detectAndCompute(frames[-1], None)
    keypoints2, descriptors2 = orb.detectAndCompute(frames_arr[i], None)
    
   
    
    matches = bf.match(descriptors1, descriptors2)
    
    #matches = sorted(matches, key=lambda x: x.distance)
    
    if len(matches) < 300:
        frames.append(frames_arr[i])
print(len(frames))

In [ ]:
%%time
detected_faces = []
count = 1
for frame in frames[:50]:
    obj = m.detect_faces(frame)
    if isinstance(obj, dict):
        for key in obj.keys():
            faceid = obj[key]
            area = faceid['facial_area']
            x1,y1,x2,y2=area
            extracted_face=frame[y1:y2,x1:x2]
            extracted_face=cv2.cvtColor(extracted_face,cv2.COLOR_BGR2RGB)
            detected_faces.append(extracted_face)
print(len(detected_faces))
plt.imshow(detected_faces[0])

In [ ]:
faces = []
faces.append(cv2.convertScaleAbs(detected_faces[0]))

for i in range(1, len(detected_faces)):
    img2 = cv2.convertScaleAbs(detected_faces[i])

    orb = cv2.ORB_create()

    keypoints1, descriptors1 = orb.detectAndCompute(faces[-1], None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

    if descriptors2 is None:
        
        continue

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)

    matches = sorted(matches, key=lambda x: x.distance)

    if len(matches) < 100:
        faces.append(detected_faces[i])

detected_faces[:]=faces[:]
print(len(detected_faces))

plt.imshow(detected_faces[2])

In [ ]:
%%time
models=[]
res=[]

for i in range(len(detected_faces)):
    model=df.find(img_path=detected_faces[i],db_path='Data/known_faces',model_name="Facenet",distance_metric="euclidean",enforce_detection=False,normalization="Facenet")
    models.append(model)

In [ ]:
%%time
count = 0
for model in models:
    if len(model[0]) > 0:
        print(count , "_ ", model[0]['identity'].values[0][104:-4])
    else:
        print('Unknown Face detected')
    count += 1

In [ ]:
%%time
models=[]
res=[]

for i in range(len(detected_faces)):
    model=df.find(img_path="Data\known_faces\Ameeza\Ameeza1.jpg",db_path='Data/known_faces',model_name="Facenet",distance_metric="euclidean",enforce_detection=False,normalization="Facenet")
    models.append(model)

In [ ]:
models[0]